In [1]:
import pandas as pd

csv_file_path = 'Dataset_v6.csv'

df = pd.read_csv(csv_file_path)

df.head()

c:\Users\13980611\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


,Id,Hospital,Type,Comment,CleanedComment,ExperienceDate,ExperienceDateString,Valence,Unit,Code,...,CovidRelated,CovidPeriod,ProcessedUnit,Gender,Ethnicity,Age,Income/Poverty Level,Employment Status,Access to Transportation,Educational Attainment
0,106,Almonte General Hospital,Inpatient,"Compassionate care, such professional staff fr...",compassionate professional staff emergency adm...,2021-02-01 00:00:00+00:00,2/1/2021,Positive,NaN,NaN,...,False,True,Unknown,Female,White or Caucasian,45-64,Below FPL,Employed,Public Transport,Some College
1,430,Almonte General Hospital,Inpatient,I was very pleased with the care & concern giv...,please given thankful health absolutely commen...,2020-06-01 00:00:00+00:00,6/1/2020,Positive,NaN,NaN,...,False,True,Unknown,Female,White or Caucasian,45-64,200-399% FPL,Employed,Limited/No Access,Less than HS
2,650,Almonte General Hospital,Inpatient,Thankfully I was admitted into the last surplu...,admit surplus wsa private washroom button fabu...,2017-08-01 00:00:00+00:00,8/1/2017,Positive,NaN,NaN,...,False,False,Unknown,Female,White or Caucasian,25-44,100-199% FPL,Unemployed,Personal Vehicle,Bachelor's+
3,715,Almonte General Hospital,Inpatient,The nurses and doctors made giving birth durin...,staff staff birth pandemic possible given circ...,2020-05-01 00:00:00+00:00,5/1/2020,Positive,NaN,NaN,...,True,True,Unknown,Male,White or Caucasian,0-14,400% or more FPL,Not in Labor Force,Public Transport,Less than HS
4,719,Almonte General Hospital,Inpatient,The nurses at the hospital were amazing! Could...,staff thankful,2018-06-01 00:00:00+00:00,6/1/2018,Positive,NaN,NaN,...,False,False,Unknown,Male,White or Caucasian,0-14,Below FPL,Not in Labor Force,Limited/No Access,Less than HS


In [75]:
def covidPeriodCheck(row):
    year = row['Year']
    if year == 2021 or year == 2020 or year == 2022:
        return True
    else:
        return False

In [77]:
from tqdm import tqdm

tqdm.pandas()

df['CovidPeriod'] = df.progress_apply(covidPeriodCheck, axis=1)

100%|██████████| 122196/122196 [00:00<00:00, 135586.54it/s]


In [80]:
def getUnitDict(CovidPeriod):
    negative_dict = {}
    positive_dict = {}
    neutral_dict = {}
    posneg_dict = {}
    total_dict = {}
    
    for index, row in df.iterrows():
        if row['Year'] == 2023:
             continue
        if row['CovidPeriod'] == CovidPeriod:
            unit =  str(row['ProcessedUnit2'])

            if unit not in total_dict.keys():
                total_dict[unit] = 1
            elif unit in total_dict.keys():
                total_dict[unit] += 1

            if row['RefinedValence'] == "Negative":
                    if unit not in negative_dict.keys():
                        negative_dict[unit] = 1
                    elif unit in negative_dict.keys():
                        negative_dict[unit] += 1
            elif row['RefinedValence'] == "Positive":
                    if unit not in positive_dict.keys():
                        positive_dict[unit] = 1
                    elif unit in positive_dict.keys():
                        positive_dict[unit] += 1
            elif row['RefinedValence'] == "Neutral":
                    if unit not in neutral_dict.keys():
                        neutral_dict[unit] = 1
                    elif unit in neutral_dict.keys():
                        neutral_dict[unit] += 1
            else:
                    if unit not in posneg_dict.keys():
                        posneg_dict[unit] = 1
                    elif unit in posneg_dict.keys():
                        posneg_dict[unit] += 1

    for unit in negative_dict.keys():
         negative_dict[unit] = round((negative_dict[unit] / total_dict[unit]) * 100, 2)
    
    for unit in positive_dict.keys():
         positive_dict[unit] = round((positive_dict[unit] / total_dict[unit]) * 100, 2)


    return dict(sorted(negative_dict.items(), key=lambda item: item[0])), dict(sorted(positive_dict.items(), key=lambda item: item[0])), neutral_dict, posneg_dict

In [81]:
negative_dict, positive_dict, neutral_dict, posneg_dict = getUnitDict(True)

In [82]:
negative_dict

{'Cardiology Unit': 24.29,
 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)': 17.57,
 'Day Surgery Unit': 18.46,
 'Dentistry': 31.25,
 'Dialysis (Renal) Unit': 15.38,
 'Emergency Department / Accident and Emergency (A&E) / Casualty Department': 32.89,
 'Gynecology Unit': 14.55,
 'Inpatient Ward': 26.47,
 'Medicine Unit': 32.14,
 'Mental Health': 42.17,
 'Neurology Unit': 31.82,
 'Obstetrics Unit': 23.4,
 'Oncology / Cancer Center': 23.26,
 'Ophthalmology Unit': 23.53,
 'Orthopedics Unit': 31.99,
 'Outpatient': 24.24,
 'Pediatrics Unit': 25.84,
 'Radiology Unit': 31.58,
 'Rehabilitation Unit': 23.87,
 'Respiratory Unit': 18.84,
 'Unknown': 30.55,
 'Urology Unit': 30.26}

In [83]:
positive_dict

{'Cardiology Unit': 54.08,
 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)': 63.51,
 'Day Surgery Unit': 65.52,
 'Dentistry': 55.0,
 'Dialysis (Renal) Unit': 66.15,
 'Emergency Department / Accident and Emergency (A&E) / Casualty Department': 42.34,
 'Gynecology Unit': 47.27,
 'Inpatient Ward': 49.27,
 'Medicine Unit': 43.72,
 'Mental Health': 41.77,
 'Neurology Unit': 46.36,
 'Obstetrics Unit': 48.76,
 'Oncology / Cancer Center': 56.07,
 'Ophthalmology Unit': 59.22,
 'Orthopedics Unit': 46.11,
 'Outpatient': 54.17,
 'Pediatrics Unit': 47.69,
 'Radiology Unit': 47.37,
 'Rehabilitation Unit': 54.41,
 'Respiratory Unit': 57.97,
 'Unknown': 46.86,
 'Urology Unit': 49.23}

In [84]:
negative_dict, positive_dict, neutral_dict, posneg_dict = getUnitDict(False)

In [87]:
negative_dict

{'Cardiology Unit': 29.04,
 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)': 20.0,
 'Day Surgery Unit': 27.42,
 'Dentistry': 20.57,
 'Dialysis (Renal) Unit': 19.05,
 'Emergency Department / Accident and Emergency (A&E) / Casualty Department': 39.5,
 'Gynecology Unit': 24.64,
 'Inpatient Ward': 32.45,
 'Medicine Unit': 40.99,
 'Mental Health': 43.45,
 'Neurology Unit': 33.23,
 'Obstetrics Unit': 28.36,
 'Oncology / Cancer Center': 34.34,
 'Ophthalmology Unit': 30.95,
 'Orthopedics Unit': 36.37,
 'Outpatient': 27.63,
 'Pediatrics Unit': 32.36,
 'Radiology Unit': 26.35,
 'Rehabilitation Unit': 33.99,
 'Respiratory Unit': 28.15,
 'Unknown': 36.95,
 'Urology Unit': 32.16}

In [88]:
positive_dict

{'Burn Unit / Burn Center': 87.5,
 'Cardiology Unit': 50.16,
 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)': 62.73,
 'Day Surgery Unit': 55.18,
 'Dentistry': 63.12,
 'Dialysis (Renal) Unit': 47.62,
 'Emergency Department / Accident and Emergency (A&E) / Casualty Department': 35.98,
 'Gynecology Unit': 47.83,
 'Inpatient Ward': 43.65,
 'Medicine Unit': 37.12,
 'Mental Health': 34.46,
 'Neurology Unit': 41.09,
 'Obstetrics Unit': 38.85,
 'Oncology / Cancer Center': 41.06,
 'Ophthalmology Unit': 47.92,
 'Orthopedics Unit': 42.13,
 'Outpatient': 53.86,
 'Pediatrics Unit': 42.24,
 'Radiology Unit': 47.3,
 'Rehabilitation Unit': 45.07,
 'Respiratory Unit': 47.41,
 'Unknown': 40.0,
 'Urology Unit': 49.12}

In [89]:
def getCounts():
    review_counts = {}

    for index, row in df.iterrows():
        year = row["Year"]

        if year == 2023:
            continue

        if year not in review_counts:
            review_counts[year] = 1
        else:
            review_counts[year] += 1

    return review_counts

In [90]:
getCounts()

{2021: 16803,
 2020: 26243,
 2017: 19694,
 2018: 24613,
 2019: 24148,
 2022: 1708,
 2016: 864,
 2015: 541}

In [91]:
def getSentimentCounts():
    positive = {}
    negative = {}
    neutral = {}
    posneg = {}

    for index, row in df.iterrows():
        year = row["Year"]

        if year == 2023:
            continue

        if row['RefinedValence'] == "Positive":
            if year not in positive:
                positive[year] = 1
            else:
                positive[year] += 1
        elif row['RefinedValence'] == "Negative":
            if year not in negative:
                negative[year] = 1
            else:
                negative[year] += 1
        elif row['RefinedValence'] == "Neutral":
            if year not in neutral:
                neutral[year] = 1
            else:
                neutral[year] += 1
        else:
            if year not in posneg:
                posneg[year] = 1
            else:
                posneg[year] += 1
        

    return positive, negative, neutral, posneg

In [92]:
positive, negative, neutral, posneg = getSentimentCounts()

In [93]:
posneg = dict(sorted(posneg.items(), key=lambda item: item[0]))

In [94]:
posneg.keys()

dict_keys([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [95]:
posneg.values()

dict_values([71, 181, 3840, 4813, 4682, 4951, 3372, 339])

In [96]:
positive = dict(sorted(positive.items(), key=lambda item: item[0]))
positive.values()

dict_values([304, 427, 8044, 10164, 10165, 12517, 7875, 782])

In [97]:
negative = dict(sorted(negative.items(), key=lambda item: item[0]))
negative.values()

dict_values([135, 214, 6917, 8690, 8471, 7639, 4890, 537])

In [98]:
neutral = dict(sorted(neutral.items(), key=lambda item: item[0]))
neutral.values()

dict_values([31, 42, 893, 946, 830, 1136, 666, 50])

In [108]:
def getTypeDict():
    negative_dict = {}
    positive_dict = {}

    
    for index, row in df.iterrows():
        year = row["Year"]
        if year == 2023:
             continue
        if str(row['Refined_Type']) == "nan":
                continue
        ref_type = row['Refined_Type']

        if row['RefinedValence'] == "Negative":
                if year not in negative_dict.keys():
                     negative_dict[year] = {}
                     
                if ref_type not in negative_dict[year].keys():
                    negative_dict[year][ref_type] = 1
                elif ref_type in negative_dict[year].keys():
                    negative_dict[year][ref_type] += 1
        elif row['RefinedValence'] == "Positive":
                if year not in positive_dict.keys():
                     positive_dict[year] = {}
                     
                if ref_type not in positive_dict[year].keys():
                    positive_dict[year][ref_type] = 1
                elif ref_type in positive_dict[year].keys():
                    positive_dict[year][ref_type] += 1


    return negative_dict, positive_dict

In [109]:
negative_dict, positive_dict = getTypeDict()

In [110]:
negative_dict

{2019: {'Inpatient': 1189,
  'General': 4628,
  'Service Alert': 553,
  'Emergency': 1438,
  'Medical/Surgical': 66,
  'Maternity': 26,
  'Psychiatric': 234,
  'Pediatric': 88,
  'Rehabilitation': 59,
  'Day Surgery': 32},
 2018: {'General': 4786,
  'Service Alert': 561,
  'Emergency': 1387,
  'Medical/Surgical': 46,
  'Inpatient': 1226,
  'Outpatient': 11,
  'Maternity': 56,
  'Day Surgery': 41,
  'Psychiatric': 246,
  'Pediatric': 107,
  'Rehabilitation': 57},
 2021: {'General': 3084,
  'Service Alert': 375,
  'Emergency': 745,
  'Outpatient': 3,
  'Medical/Surgical': 22,
  'Inpatient': 424,
  'Psychiatric': 81,
  'Pediatric': 20,
  'Maternity': 5,
  'Day Surgery': 8},
 2020: {'General': 4107,
  'Service Alert': 524,
  'Emergency': 1444,
  'Medical/Surgical': 56,
  'Inpatient': 980,
  'Psychiatric': 202,
  'Pediatric': 73,
  'Day Surgery': 39,
  'Maternity': 12},
 2017: {'General': 3969,
  'Service Alert': 427,
  'Emergency': 1013,
  'Medical/Surgical': 46,
  'Psychiatric': 188,
  'P

In [111]:
positive_dict

{2021: {'Inpatient': 635,
  'General': 5958,
  'Emergency': 807,
  'Outpatient': 10,
  'Medical/Surgical': 45,
  'Service Alert': 4,
  'Psychiatric': 177,
  'Pediatric': 45,
  'Day Surgery': 64,
  'Maternity': 17},
 2020: {'Inpatient': 1483,
  'General': 8145,
  'Service Alert': 12,
  'Emergency': 1749,
  'Medical/Surgical': 129,
  'Psychiatric': 393,
  'Pediatric': 141,
  'Day Surgery': 191,
  'Maternity': 49},
 2017: {'Inpatient': 972,
  'General': 5231,
  'Service Alert': 17,
  'Emergency': 803,
  'Medical/Surgical': 108,
  'Psychiatric': 266,
  'Pediatric': 151,
  'Maternity': 95,
  'Rehabilitation': 82,
  'Day Surgery': 131},
 2018: {'Inpatient': 1409,
  'General': 6563,
  'Service Alert': 10,
  'Emergency': 1077,
  'Medical/Surgical': 110,
  'Outpatient': 13,
  'Maternity': 108,
  'Day Surgery': 136,
  'Psychiatric': 360,
  'Pediatric': 98,
  'Rehabilitation': 101},
 2019: {'Inpatient': 1237,
  'General': 6795,
  'Emergency': 1085,
  'Medical/Surgical': 107,
  'Maternity': 52,
  

In [134]:
import re

def remove_substrings(s):
    pattern = r'x{3,}'
    return re.sub(pattern, '', s)

In [152]:
import ast

def getEntityDict(year):
    positive_dict = {}
    negative_dict = {}

    for index, row in df.iterrows():
        if row['Year'] == year:
            if row['RefinedValence'] == "Positive/Negative" or row['RefinedValence'] == "Positive":
                
                for entity, label in ast.literal_eval(row['Entities']):
                    if ((entity not in positive_dict.keys()) and (not entity.startswith('xx'))):
                        positive_dict[entity] = 1
                    elif entity in positive_dict.keys():
                        positive_dict[entity] += 1
            elif row['RefinedValence'] == "Positive/Negative" or row['RefinedValence'] == "Negative":
                for entity, label in ast.literal_eval(row['Entities']):
                    if ((entity not in negative_dict.keys()) and (not entity.startswith('xx'))):
                        negative_dict[entity] = 1
                    elif entity in negative_dict.keys():
                        negative_dict[entity] += 1
        
    cleaned_positive_dict = {remove_substrings(k): v for k, v in positive_dict.items()}
    sorted_positive_dict = dict(sorted(cleaned_positive_dict.items(), key=lambda item: item[1]))

    cleaned_negative_dict = {remove_substrings(k): v for k, v in negative_dict.items()}
    sorted_negative_dict = dict(sorted(cleaned_negative_dict.items(), key=lambda item: item[1]))

    return sorted_positive_dict, sorted_negative_dict

In [160]:
sorted_positive_dict, sorted_negative_dict = getEntityDict(2021)
    
top_10 = sorted(sorted_negative_dict.items(), key=lambda item: item[1], reverse=True)[:10]

# Rank the top 10 items from 1 to 10
ranked_top_10 = {rank: key for rank, (key, value) in enumerate(top_10, start=1)}

print(ranked_top_10)

{1: 'pain', 2: 'infection', 3: 'anxiety', 4: 'bleeding', 5: 'stroke', 6: 'fracture', 7: 'fever', 8: 'cancer', 9: 'pneumonia', 10: 'tylenol'}


In [161]:
top_10

[('pain', 669),
 ('infection', 51),
 ('anxiety', 42),
 ('bleeding', 41),
 ('stroke', 39),
 ('fracture', 37),
 ('fever', 33),
 ('cancer', 26),
 ('pneumonia', 24),
 ('tylenol', 21)]

In [162]:
def getTypeDictPerc(CovidPeriod):
    negative_dict = {}
    positive_dict = {}
    total_dict = {}

    
    for index, row in df.iterrows():
        if row['CovidPeriod'] == CovidPeriod:
            if str(row['Refined_Type']) == "nan":
                    continue
            ref_type = row['Refined_Type']

            if ref_type not in total_dict.keys():
                total_dict[ref_type] = 1
            elif ref_type in total_dict.keys():
                total_dict[ref_type] += 1


            if row['RefinedValence'] == "Negative":
                
                    if ref_type not in negative_dict.keys():
                        negative_dict[ref_type] = 1
                    elif ref_type in negative_dict.keys():
                        negative_dict[ref_type] += 1
            elif row['RefinedValence'] == "Positive":
                    if ref_type not in positive_dict.keys():
                        positive_dict[ref_type] = 1
                    elif ref_type in positive_dict.keys():
                        positive_dict[ref_type] += 1

    for ref_type in negative_dict.keys():
         negative_dict[ref_type] = round((negative_dict[ref_type] / total_dict[ref_type]) * 100, 2)
    
    for ref_type in positive_dict.keys():
         positive_dict[ref_type] = round((positive_dict[ref_type] / total_dict[ref_type]) * 100, 2)

    return negative_dict, positive_dict

In [163]:
negative_dict, positive_dict = getTypeDictPerc(True)

In [164]:
sorted_dict = dict(sorted(negative_dict.items(), key=lambda item: item[1]))

sorted_dict

{'Outpatient': 12.5,
 'Day Surgery': 13.31,
 'Maternity': 17.71,
 'Medical/Surgical': 24.61,
 'Pediatric': 25.62,
 'Psychiatric': 25.95,
 'General': 26.23,
 'Inpatient': 30.88,
 'Emergency': 34.74,
 'Service Alert': 63.57}

In [165]:
sorted_dict = dict(sorted(positive_dict.items(), key=lambda item: item[1]))

sorted_dict

{'Service Alert': 1.22,
 'Emergency': 40.49,
 'Outpatient': 41.67,
 'Inpatient': 46.59,
 'General': 50.91,
 'Psychiatric': 51.12,
 'Pediatric': 51.24,
 'Medical/Surgical': 54.89,
 'Maternity': 68.75,
 'Day Surgery': 72.24}

In [166]:
negative_dict, positive_dict = getTypeDictPerc(False)

In [167]:
sorted_dict = dict(sorted(negative_dict.items(), key=lambda item: item[1]))

sorted_dict

{'Day Surgery': 17.98,
 'Maternity': 22.47,
 'Medical/Surgical': 26.73,
 'Rehabilitation': 30.84,
 'Psychiatric': 31.23,
 'General': 31.53,
 'Pediatric': 35.31,
 'Inpatient': 36.42,
 'Outpatient': 36.67,
 'Emergency': 43.1,
 'Service Alert': 65.52}

In [168]:
sorted_dict = dict(sorted(positive_dict.items(), key=lambda item: item[1]))

sorted_dict

{'Service Alert': 1.3,
 'Emergency': 33.3,
 'Inpatient': 40.07,
 'Maternity': 43.07,
 'Pediatric': 43.2,
 'Outpatient': 43.33,
 'General': 45.34,
 'Psychiatric': 46.24,
 'Rehabilitation': 48.26,
 'Medical/Surgical': 54.99,
 'Day Surgery': 70.55}

In [ ]:
#######################################3

In [100]:
def getSentimentPercents():
    positive = {}
    negative = {}
    neutral = {}
    posneg = {}
    total = {}

    for index, row in df.iterrows():
        year = row["Year"]

        if year == 2023:
            continue

        if year not in total:
            total[year] = 1
        else:
            total[year] += 1

        if row['RefinedValence'] == "Positive":
            if year not in positive:
                positive[year] = 1
            else:
                positive[year] += 1
        elif row['RefinedValence'] == "Negative":
            if year not in negative:
                negative[year] = 1
            else:
                negative[year] += 1
        elif row['RefinedValence'] == "Neutral":
            if year not in neutral:
                neutral[year] = 1
            else:
                neutral[year] += 1
        else:
            if year not in posneg:
                posneg[year] = 1
            else:
                posneg[year] += 1
        
    for year in positive.keys():
         positive[year] = round((positive[year] / total[year]) * 100, 2)
    
    for year in negative.keys():
         negative[year] = round((negative[year] / total[year]) * 100, 2)

    for year in neutral.keys():
         neutral[year] = round((neutral[year] / total[year]) * 100, 2)

    for year in posneg.keys():
         posneg[year] = round((posneg[year] / total[year]) * 100, 2)

    return positive, negative, neutral, posneg

In [101]:
positive, negative, neutral, posneg = getSentimentPercents()

In [102]:
posneg = dict(sorted(posneg.items(), key=lambda item: item[0]))

In [103]:
posneg.keys()

dict_keys([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [104]:
posneg.values()

dict_values([13.12, 20.95, 19.5, 19.55, 19.39, 18.87, 20.07, 19.85])

In [105]:
positive = dict(sorted(positive.items(), key=lambda item: item[0]))
positive.values()

dict_values([56.19, 49.42, 40.84, 41.3, 42.09, 47.7, 46.87, 45.78])

In [106]:
negative = dict(sorted(negative.items(), key=lambda item: item[0]))
negative.values()

dict_values([24.95, 24.77, 35.12, 35.31, 35.08, 29.11, 29.1, 31.44])

In [107]:
neutral = dict(sorted(neutral.items(), key=lambda item: item[0]))
neutral.values()

dict_values([5.73, 4.86, 4.53, 3.84, 3.44, 4.33, 3.96, 2.93])

In [ ]:
filtered_df = df[df['Hospital'] == 'SickKids Hospital']
filtered_df['ProcessedUnit'] = filtered_df['Unit'].map(unit_mapping).fillna(df['Unit'])

# Print the DataFrame to see the new "ProcessedUnit" column
print(filtered_df)

In [88]:
for hospital, group in df.groupby('Hospital'):
    print(f"Unique Units for {hospital}:")
    unique_units = group['Unit'].unique()
    print(unique_units)
    print("\n")

Unique Units for Almonte General Hospital:
[nan]


Unique Units for Bluewater:
['Medicine G' 'Medicine' 'Medicine T Telemetry' 'Surgery'
 'Emergency Sarnia' 'Rehabilitation' 'Med/Surg - Rural Health (CEE)'
 'Obstetrics' 'Surgery B' 'Emergency Rural Health' 'Day Surgery'
 'Medicine A' 'Acute Mental Health Inpatient' 'Surgery A']


Unique Units for Brant Community Healthcare System:
['Brantford Site' 'Ucare TWH' 'Fracture Clinic' 'Ambulatory' 'OP Stroke'
 'DEC BGH' 'DEC-TWH']


Unique Units for Campbellford:
['Emergency' 'Inpatient']


Unique Units for Children's Hospital of Eastern Ontario:
['5 East' 'CHEO. MH OUT-PATIENT SERVICES•6TH FLOOR' '4 West'
 'CHEO MH OUT-PATIENT SERVICES 6TH FLOOR.'
 'CHEO MH OUT-PATIENT SERVICES 6TH FLOOR'
 'CHEO MU OUT-PATIENT SERVICES MONTREAL RD'
 'EATING DISORDERS INPATIENT PROGRAM CHEO 6'
 'INPATIENT PSYCHIATRY PROGRAM CHEO 6E' 'CHEO MH EATING DISORDER 5 NORTH'
 'CHEO MH OUT-PATIENT SERVICES MONTREAL RD' '4 North'
 'CHEO. MFI OUT-PATIENT SERVICES 6TH FLO

In [62]:
unit_mapping = {
'C2 (Email)': 'Inpatient Ward',
'D3 (Mail)': 'Inpatient Ward',
'16E (Mail)': 'Inpatient Ward',
'13E (Mail)': 'Inpatient Ward',
'Emergency (Mail)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'xxMSK; A4': 'Orthopedics Unit',
'A2': 'Inpatient Ward',
'C4 (Mail)': 'Inpatient Ward',
'17E (Mail)': 'Inpatient Ward', 
'D2 (Mail)': 'Inpatient Ward',
'D5 (Mail)': 'Inpatient Ward',
'A1': 'Inpatient Ward',
'C5 (Mail)': 'Inpatient Ward',
'xxTrauma': 'Mental Health',
'C6': 'Inpatient Ward',
'xxCardiac': 'Cardiology Unit',
'C3/CCU (Mail)': 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)',
'B4 (Mail)': 'Inpatient Ward',
'A3': 'Inpatient Ward',
'xxS.T.A.R.': 'Rehabilitation Unit',
'M5LB/M5NC/M5NW/M5NE': 'Obstetrics Unit',
'CVIC': 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)',
'M5MN': 'Obstetrics Unit',
'M5HR': 'Obstetrics Unit',
'D4 (Mail)': 'Inpatient Ward',
'Emergency (Email)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department', 
'B5 IC': 'Inpatient Ward',
'C2 (Mail)': 'Inpatient Ward',
'D6 (Mail)': 'Inpatient Ward',
'A4': 'Inpatient Ward',
'C5 (Email)': 'Inpatient Ward',
'D5 (Email)': 'Inpatient Ward',
'C4 (Email)': 'Inpatient Ward',
'D6 (Email)': 'Inpatient Ward',
'xxOncology': 'Oncology / Cancer Center', 
'xxNeurology': 'Neurology Unit',
'D3 (Email)': 'Inpatient Ward',
'xxAmputee': 'Rehabilitation Unit',
'C3/CCU (Email)': 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)',
'C6 (Email)': 'Inpatient Ward',
'13E (Email)': 'Inpatient Ward',
'xxBurns': 'Burn Unit / Burn Center', 
'16E (Email)': 'Inpatient Ward',
'B4 (Email)': 'Inpatient Ward',
'D2 (Email)': 'Inpatient Ward',
'Day Surgery (Mail)' : 'Day Surgery Unit',
'Surgery (Mail)': 'Inpatient Ward',
'Woodstock General Hospital; Day Surgery' : 'Day Surgery Unit',
'Day Surgery; Woodstock General Hospital' : 'Day Surgery Unit',
'Medical (Mail)': 'Medicine Unit',
'Medical; Woodstock General Hospital': 'Medicine Unit',
'Surgery; Woodstock General Hospital': 'Inpatient Ward',
'Woodstock General Hospital; Surgery': 'Inpatient Ward',
'Woodstock General Hospital; Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'CCC Rehab (Handout)': 'Rehabilitation Unit',
'Emergency; Woodstock General Hospital': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Rehabilitation; Woodstock General Hospital': 'Rehabilitation Unit',
'Woodstock General Hospital; Medical' : 'Medicine Unit',
'Continuing Complex Care Rehab; Woodstock General Hospital': 'Rehabilitation Unit',
'Woodstock General Hospital; Rehabilitation': 'Rehabilitation Unit',
'Woodstock General Hospital; Continuing Complex Care Rehab': 'Rehabilitation Unit',
'Cardiac: CARU/2C (Combined), MH (Mail)': 'Cardiology Unit',
'Surgery: SURG2B, MH (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Cardiac: CSPU2J, MH (Mail)': 'Cardiology Unit',
'Neuro/MSK: ORTH1J, MH (Mail)': 'Orthopedics Unit',
'Surgery: SURG2B2, MH (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Surgery: 1B Surg, CVH (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Medicine: 3A3 Flex, MH (Mail)': 'Medicine Unit',
'Medicine: 1A, CVH (Mail)': 'Medicine Unit',
'Oncology: MED 6J, MH (Mail)': 'Oncology / Cancer Center',
'Cardiac: 3B Cardiology, CVH (Mail)': 'Cardiology Unit',
'Oncology: ONCIN, CVH (Mail)': 'Oncology / Cancer Center',
'Medicine: 2B GMed, CVH (Mail)': 'Medicine Unit',
'Medicine: 1E Med, CVH (Mail)': 'Medicine Unit',
'Medicine: 4B, MH (Mail)': 'Medicine Unit',
'M-MED4D (Phone)': 'Medicine Unit',
'Cardiac: CCU 3J, MH (Mail)': 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)',
'Cardiac: Short Stay CSSU2D, MH (Mail)': 'Cardiology Unit',
'Neuro/MSK: Surgical Spine Unit, MH (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit for spine/neurosurgery
'Neuro/MSK: NEUS4D, MH (Mail)': 'Neurology Unit',
'M-CSA3A (Phone)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Cardiac: 3B Cardiology, CVH (Phone)': 'Cardiology Unit',
'Medicine: 2C GMed, CVH (Mail)': 'Medicine Unit',
'Medicine: 2B GMed, CVH (Phone)': 'Medicine Unit',
'Cardiac: Short Stay CSSU2D, MH (Phone)': 'Cardiology Unit',
'Medicine: 5J, MH (Mail)': 'Medicine Unit',
'Medicine: 1C/3B Med, CVH (Mail)': 'Medicine Unit',
'1A-SURG, CVH (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Cardiac: CARU/2C (Combined), MH (Phone)': 'Cardiology Unit',
'Medicine: 1E Med, CVH (Phone)': 'Medicine Unit',
'Medicine: 1C, CVH (Phone)': 'Medicine Unit',
'M-CCU (Phone)': 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)',
'Medicine: 3A, MH (Phone)': 'Medicine Unit',
'M-NSSU4J (Phone)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Neuro/MSK: ORTH1J, MH (Phone)': 'Orthopedics Unit',
'1B MED, CVH (Phone)': 'Medicine Unit',
'Medicine: MED4B, MH (Phone)': 'Medicine Unit',
'Oncology: ONCIN, CVH (Phone)': 'Oncology / Cancer Center',
'Surgery: SURG2B, MH (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Intensive Care Unit, MH (Mail)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Cardiac: CSPU2J, MH (Phone)': 'Cardiology Unit',
'Oncology: MED6J, MH (Phone)': 'Oncology / Cancer Center',
'Intensive Care Unit, CVH (Mail)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'M-NEUS4J (Phone)': 'Neurology Unit',
'Medicine: 2C GMed, CVH (Phone)': 'Medicine Unit',
'Medicine: 3A/3A2, MH (Mail)': 'Medicine Unit',
'Medicine: MED5J, MH (Phone)': 'Medicine Unit',
'Neuro/MSK: Comp. Stroke Unit: NSUA4C, MH (Mail)': 'Neurology Unit',
'Surgery: SURG2B2, MH (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'M-Med4C (Phone)': 'Medicine Unit',
'C CON 2A SURGE, CVH (Phone)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Met - Medicine ME2 (Mail)': 'Medicine Unit',
'Met - Medicine ON1 (Mail)': 'Medicine Unit',
'EDPEC - Ouellette (Mail)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'EDPEC - Metropolitan (Mail)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Oue-Surgery 6E (Email)': 'Inpatient Ward', # Assuming this is a surgical unit
'Met - Medicine SSM (Mail)': 'Medicine Unit',
'Maternity - Metropolitan (Mail)': 'Obstetrics Unit',
'EDPEC - Metropolitan (Email)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Oue-Surgery 8E Neurosurgery (Mail)': 'Inpatient Ward', # Assuming this is a neurosurgical unit
'Met- Surgery SU1 (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Met- Medicine ME2 (Email)': 'Medicine Unit',
'Phone (CIHI Mode)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Met- Surgery SU2 (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Maternity - Metropolitan (Email)': 'Obstetrics Unit',
'Oue-Surgery 8E Neurosurgery (Email)': 'Inpatient Ward', # Assuming this is a neurosurgical unit
'Oue - Medicine 7E (Mail)': 'Medicine Unit',
'Met- Medicine ME3 (Email)': 'Medicine Unit',
'Oue-Surgery6E (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Oue-Medicine 5A (Email)': 'Medicine Unit',
'Oue-Surgery 8W Neurology (Email)': 'Inpatient Ward', # Assuming this is a neurology unit
'Oue-Surgery8ENeurosurgery (Email)': 'Inpatient Ward', # Assuming this is a neurosurgical unit
'EDPEC - Ouellette (Email)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Met- Medicine SSM (Email)': 'Medicine Unit',
'Oue-Surgery 6E (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Oue-Surgery 6W (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Oue-Surgery 6W (Email)': 'Inpatient Ward', # Assuming this is a surgical unit
'Oue - Medicine 5A (Mail)': 'Medicine Unit',
'Met-Surgery SU1 (Email)': 'Inpatient Ward', # Assuming this is a surgical unit
'CPES-IC': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Oue-Surgery 8W Neurology (Mail)': 'Inpatient Ward', # Assuming this is a neurology unit
'Oue-Surgery6E (Email)': 'Inpatient Ward', # Assuming this is a surgical unit
'Oue - Medicine 7W (Mail)': 'Medicine Unit',
'Met - Medicine ME3 (Mail)': 'Medicine Unit',
'PPEC - Metropolitan (Mail)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department', # Assuming PPEC stands for Emergency Department
'Metropolitan - SU1 (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Met-Surgery SU1 (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Phone Discharge (CIHI Mode)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Met-Surgery SU2 (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Oue-Surgery6W (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Oue - Medicine 4 (Mail)': 'Medicine Unit',
'Met-MedicineME3 (Email)': 'Medicine Unit',
'Phone Emergency Dept, (CIHI Emotional Support, Mode) Physical Comfort, Infection Prevention & Control': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Oue-Surgery8ENeurosurgery (Mail)': 'Inpatient Ward', # Assuming this is a neurosurgical unit
'Phone General Comment (CIHI Mode)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Ouellette - 6th Floor W (Phone)': 'Inpatient Ward', # Assuming this is an inpatient unit
'Met - Medicine ME3 (Email)': 'Medicine Unit',
'Metropolitan - ME3 (Phone)': 'Medicine Unit',
'Met - MedicineSSM (Mail)': 'Medicine Unit',
'Met- Medicine ON1 (Email)': 'Medicine Unit',
'Met - MedicineON1 (Mail)': 'Medicine Unit',
'Maternity -Metropolitan (Email)': 'Obstetrics Unit',
'Phone Respect to Patient (CIHI Mode)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Met- Surgery SU2 (Email)': 'Inpatient Ward', # Assuming this is a surgical unit
'Ouellette - 8E Neurosurg (Phone)': 'Inpatient Ward', # Assuming this is a neurosurgical unit
'PPEC - Metropolitan (Email)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'CPES-IC ': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Maternity -': 'Obstetrics Unit',
'Oue-Surgery 8ENeurosurgery (Mail)': 'Inpatient Ward', # Assuming this is a neurosurgical unit
'Phone Physical Comfort, (CIHI Respect to Patient Mode)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Oue-Surgery8WNeurology (Mail)': 'Inpatient Ward', # Assuming this is a neurology unit
'Oue-Surgery 8ENeurosurgery (Email)': 'Inpatient Ward', # Assuming this is a neurosurgical unit
'Phone Emotional Support (CIHI Mode)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Met-Surgery SU2 (Email)': 'Inpatient Ward', # Assuming this is a surgical unit
'Metropolitan - SU2 (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Met- Surgery': 'Inpatient Ward', # Assuming this is a surgical unit
'Metropolitan - SSM (Phone)': 'Medicine Unit',
'Met - MedicineME3 (Mail)': 'Medicine Unit',
'Met- Surgery SU1 (Email)': 'Inpatient Ward', # Assuming this is a surgical unit
'Phone Housekeeping/Room (CIHI Mode)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Metropolitan - ME2 (Phone)': 'Medicine Unit',
'Phone Access/Coord of Care, (CIHI Doctor/Physician, Mode) Housekeeping/Room, Nurse/Nurse aide, Respect to Patient': 'Inpatient Ward', # Unsure, mapping to Inpatient
'Surgical IP-7 Surgical (Phone)': 'Inpatient Ward',
'Surgical IP-Chest (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Emergency (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Emergency (Mail)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'General Med (Phone)': 'Medicine Unit',
'ED: AgeGrp 35-44': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department', # Assuming ED stands for Emergency Department
'Kidney Urinary (Phone)': 'Urology Unit',
'Surgical IP-6 Surgical (Phone)': 'Inpatient Ward',
'5008912': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'ED: AgeGrp 45-54': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Kidney Urinary Nephrology (Phone)': 'Dialysis (Renal) Unit',
'General Med CTU West (Phone)': 'Medicine Unit', # Assuming CTU stands for Clinical Treatment Unit
'Surgical IP - 7 Surgical (Phone)': 'Inpatient Ward',
'CPES: AgeGrp 18-34; General Med-CTU Central (Phone)': 'Medicine Unit', # Assuming CPES stands for a unit/department
'General Med CTU Central (Phone)': 'Medicine Unit',
'Surgical IP-6 Surgical (Phone); CPES: AgeGrp 45-54': 'Inpatient Ward',
'Surgical IP-Head & Neck (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Surgical IP (Phone)': 'Inpatient Ward',
'Kidney Urinary Renal Transplant (Phone)': 'Dialysis (Renal) Unit',
'ED: AgeGrp 18-34': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'General Med CTU North (Phone)': 'Medicine Unit',
'Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'5008911': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'ED: AgeGrp 65+': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'General Med-CTU Central (Phone); CPES: AgeGrp 45-54': 'Medicine Unit',
'Surgical IP - Chest (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'General Med CTU North (Phone)': 'Medicine Unit',
'Kidney Urinary (Phone)': 'Urology Unit',
'ED: AgeGrp 55-64': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Surgical IP-Head & Neck (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Surgical IP - Head & Neck (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Surgical IP-Chest (Phone): CPES: AgeGrp 65+': 'Inpatient Ward', # Assuming this is a surgical unit
'General Med - CTU West (Phone): CPES: AgeGrp 18-34': 'Medicine Unit',
'ED: AgeGrp 45-54 ED:': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Kidney Urinary Nephrology (Phone); CPES: AgeGrp 65+': 'Dialysis (Renal) Unit',
'Surgical IP-Chest (Phone); CPES: AgeGrp 65+': 'Inpatient Ward', # Assuming this is a surgical unit
'General Med - CTU West (Phone); CPES: AgeGrp 65+': 'Medicine Unit',
'Kidney Urinary Nephrology (Phone). CPES: AgeGrp 65+': 'Dialysis (Renal) Unit',
'Surgical IP-Chest (Phone); CPES: AgeGrp 55-64': 'Inpatient Ward', # Assuming this is a surgical unit
'CPES: AgeGrp 65+: Surgical IP-7 Surgical (Phone)': 'Inpatient Ward',
'CPES: AgeGrp 55-64: General Med CTU West (Phone)': 'Medicine Unit',
'Surgical IP-6 Surgical (Phone); CPES: AgeGrp 65+': 'Inpatient Ward',
'Surgical IP-6 Surgical (Phone): CPES: AgeGrp 65+': 'Inpatient Ward',
'(Phone); CPES: AgeGrp 45-54': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Surgical IP - 6 Surgical (Phone)': 'Inpatient Ward',
'General Med - CTU West (Phone); CPES: AgeGrp 65+ Surgical IP-7 Surgical': 'Medicine Unit',
'Surgical IP - (Phone)': 'Inpatient Ward',
'Surgical IP-Chest (Phone); CPES: AgeGrp 45-54': 'Inpatient Ward', # Assuming this is a surgical unit
'Surgical IP-6 Surgical (Phone); CPES: AgeGrp 35-44': 'Inpatient Ward',
'General Med. (Phone)': 'Medicine Unit',
'General Med - CTU North (Phone); CPES: AgeGrp 55-64': 'Medicine Unit',
'Surgical IP-Chest (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'General Med (Phone)': 'Medicine Unit',
'Surgical IP-7 Surgical (Phone); CPES: AgeGrp 45-54': 'Inpatient Ward',
'General Med CTU North (Phone); CPES: AgeGrp 65+': 'Medicine Unit',
'Surgical IP-6 Surgical (Phone); CPES: AgeGrp 18-34': 'Inpatient Ward',
'Surgical IP (Phone)': 'Inpatient Ward',
'General Med - CTU North (Phone); CPES: AgeGrp 45-54': 'Medicine Unit',
'General Med - CTU North (Phone); CPES: AgeGrp 65+': 'Medicine Unit',
'Kidney Urinary Nephrology (Phone) CPES: AgeGrp 65+': 'Dialysis (Renal) Unit',
'ED: AgeGrp 18-34': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'CPES: AgeGrp 55-64; Surgical IP-7 Surgical (Phone)': 'Inpatient Ward',
'General Med-CTU Central (Phone); CPES: AgeGrp 65+ CPES:': 'Medicine Unit',
'CPES: AgeGrp 55-64: Kidney Urinary Nephrology (Phone)': 'Dialysis (Renal) Unit',
'Surgical IP-| (Phone)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'General Med. CTU North (Phone)': 'Medicine Unit',
'Kidney Urinary - Renal Transplant (Phone): CPES: AgeGrp 65+': 'Dialysis (Renal) Unit',
'Surgical (Phone); CPES: AgeGrp 55-64': 'Inpatient Ward', # Assuming this is a surgical unit
'Kidney Urinary-Renal Transplant (Phone): CPES: AgeGrp 35-44': 'Dialysis (Renal) Unit', 
'CPES: AgeGrp 55-64; Surgical IP - Head & Neck (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Surgical IP - 7 Surgical (Phone)': 'Inpatient Ward',
'Kidney Urinary Nephrology (Phone): CPES: AgeGrp 65+': 'Dialysis (Renal) Unit',
'CPES: AgeGrp 55-64; Kidney Urinary Nephrology (Phone)': 'Dialysis (Renal) Unit',
'ED: AgeGrp 45-54': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'CPES: AgeGrp 55-64; General Med-CTU Central (Phone)': 'Medicine Unit',
'AgeGrp 18-34; Surgical IP-7 Surgical (Phone)': 'Inpatient Ward',
'General Med - CTU West (Phone): CPES: AgeGrp 35-44': 'Medicine Unit',
'General Med - CTU Central (Phone); CPES: Age Grp 35-44': 'Medicine Unit',
'Kidney Transplant (Phone): CPES: AgeGrp 35-44': 'Dialysis (Renal) Unit',
'Surgical IP-Chest (Phone): CPES: AgeGrp 55-64': 'Inpatient Ward', # Assuming this is a surgical unit
'CPES: AgeGrp 55-64; Surgical IP-7 Surgical (Phone)': 'Inpatient Ward',
'General Med CTU Central (Phone)': 'Medicine Unit',
'Kidney Urinary - Renal Transplant (Phone): CPES: AgeGrp 45-54': 'Dialysis (Renal) Unit',
'Surgical IP-6 Surgical (Phone) General Med': 'Inpatient Ward',
'CPES: AgeGrp 55-64: General Med-CTU Central (Phone)': 'Medicine Unit',
'Surgical IP': 'Inpatient Ward',
'Surgical IP - Head & Neck (Phone); CPES: AgeGrp 65+': 'Inpatient Ward', # Assuming this is a surgical unit
'Surgical IP- Head & Neck (Phone): CPES: AgeGrp 45-54': 'Inpatient Ward', # Assuming this is a surgical unit
'Surgical IP-Chest (Phone); CPES: AgeGrp 35-44': 'Inpatient Ward', # Assuming this is a surgical unit
'Kidney Urinary-Renal Transplant (Phone): CPES: AgeGrp 65+': 'Dialysis (Renal) Unit',
'General Med-CTU Central (Phone); CPES: AgeGrp 35-44': 'Medicine Unit',
'General Med - CTU West (Phone); CPES: AgeGrp 45-54': 'Medicine Unit',
'CPES: AgeGrp 55-64: Surgical IP-Head & Neck (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'General Med': 'Medicine Unit',
'Surgical IP-Head & Neck (Phone): CPES: AgeGrp 65+': 'Inpatient Ward', # Assuming this is a surgical unit
'(Phone) Surgical IP-7 Surgical (Phone)': 'Inpatient Ward',
'Surgical IP-6 Surgical (Phone); CPES: AgeGrp 55-64': 'Inpatient Ward',
'General Med - CTU West (Phone); CPES: AgeGrp 35-44': 'Medicine Unit',
'ED: AgeGrp 35-44': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Cath Lab (Mail)': 'Inpatient Ward',
'Emergency (Mail)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Medical Assessment (Mail)': 'Inpatient Ward',
'Surgical Services (Mail)': 'Inpatient Ward',
'Mental Health IP': 'Inpatient Ward', # Assuming IP stands for Inpatient
'Heart Rhythm (Mail)': 'Cardiology Unit',
'Cataract (Mail)': 'Ophthalmology Unit',
'General Day Surgery (Mail)': 'Day Surgery Unit',
'General Medicine (Mail)': 'Medicine Unit',
'MSK Acute (Mail)': 'Inpatient Ward', # Assuming MSK stands for Musculoskeletal
'Heart Rhythm': 'Cardiology Unit',
'Cardiology (Mail)': 'Cardiology Unit',
'Outpatient Rehab (Mail)': 'Rehabilitation Unit',
'Obstetrics (Mail)': 'Obstetrics Unit',
'Heart Rhythm (Phone)': 'Cardiology Unit',
'Restorative Care - Humber-Finch Site (Mail)': 'Inpatient Ward',
'Obstetrics (Phone)': 'Obstetrics Unit',
'General Medicine (Phone)': 'Medicine Unit',
'Oncology': 'Oncology / Cancer Center',
'MSK Acute (Phone)': 'Inpatient Ward', # Assuming MSK stands for Musculoskeletal
'General Day Surgery (Phone)': 'Day Surgery Unit',
'Cardiac Surgery (Mail)': 'Inpatient Ward', # Assuming this is a surgical unit
'Emergency (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Paediatrics (Mail)': 'Pediatrics Unit',
'Cataract (Phone)': 'Ophthalmology Unit',
'Cardiology (Phone)': 'Cardiology Unit',
'Paediatrics (Phone)': 'Pediatrics Unit',
'Cardiac Surgery (Phone)': 'Inpatient Ward', # Assuming this is a surgical unit
'Cath Lab (Phone)': 'Inpatient Ward',
'Surgical Services (Phone)': 'Inpatient Ward',
'Medical Assessment (Phone)': 'Inpatient Ward',
'Outpatient Rehab (Phone); xxFracture Clinic (Phone)': 'Rehabilitation Unit',
'Cataract': 'Ophthalmology Unit',
'Mental Health OP': 'Outpatient', # Assuming OP stands for Outpatient
'Fracture Clinic': 'Orthopedics Unit',
'Restorative Care - Humber-Finch Site (Phone)': 'Inpatient Ward',
'Cath Lab': 'Inpatient Ward',
'General Day Surgery': 'Day Surgery Unit',
'xxFracture Clinic (Phone); Outpatient Rehab (Phone)': 'Rehabilitation Unit',
'Mental Health IP Acute': 'Mental Health',
'Surgical (3A)': 'Inpatient Ward',
'Obstetrics (Mail)': 'Obstetrics Unit',
'Obstetrics (Email)': 'Obstetrics Unit',
'Ambulatory Clinic - General': 'Outpatient',
'Emergency Fast Track': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Core Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Medical (3B)': 'Medicine Unit',
'Medical (3C)': 'Medicine Unit',
'Oncology': 'Oncology / Cancer Center',
'Maternity (Postcard/Handout)': 'Obstetrics Unit',
'Medical (Phone)': 'Medicine Unit',
'Rehabilitation (Handout)': 'Rehabilitation Unit',
'ER': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Integrated Medicine and Rehab Services':'Rehabilitation Unit',
'Medical S1': 'Medicine Unit',
'Medical S2': 'Medicine Unit',
'Surgical C5': 'Inpatient Ward',
'SURGB3': 'Inpatient Ward',
'SURGA3': 'Inpatient Ward',
'SURGSDA': 'Inpatient Ward',
'SURG': 'Inpatient Ward',
'MED': 'Medicine Unit',
'MEDD3': 'Medicine Unit',
'MEDC3': 'Medicine Unit',
'Medical': 'Medicine Unit',
'Surgical': 'Inpatient Ward',
'G4D': 'Inpatient Ward',
'G4A-Med': 'Medicine Unit',
'Centenary': 'Inpatient Ward',  # Assuming this is a hospital/site name
'General': 'Inpatient Ward',  # Assuming this is a hospital/site name
'YC9W': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'H3C-Sur': 'Inpatient Ward',  # Assuming this is a surgical unit
'YC2BC': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'YC9CP': 'Outpatient',  # Unsure, mapping to Outpatient
'G3D': 'Inpatient Ward',
'YC5W': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'H9T/H4E': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'H2C-Neph': 'Inpatient Ward',  # Assuming this is a nephrology unit
'H4C-Med': 'Medicine Unit',
'H3CP-ORT': 'Orthopedics Unit',
'Birchmount': 'Inpatient Ward',  # Assuming this is a hospital/site name
'H6T-ONC': 'Oncology / Cancer Center',
'G3D (Last Surveyed March 2017)': 'Inpatient Ward',
'H3CP-ORT (Last Surveyed March 2017)': 'Orthopedics Unit',
'G4FBP': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'H2T-Neph': 'Inpatient Ward',  # Assuming this is a nephrology unit
'H4CPM (Last Surveyed March 2017)': 'Medicine Unit',
'G3A-SUR': 'Inpatient Ward',  # Assuming this is a surgical unit
'General Surgery': 'Inpatient Ward',  # Assuming this is a general surgical unit
'xxYC4ME': 'Medicine Unit',  # Unsure, mapping to Medicine Unit
'H9T': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'YC5E': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'H4C-MED (Last Surveyed March 2017)': 'Medicine Unit',
'H2OBS': 'Inpatient Ward',  # Assuming this is an obstetrics unit
'H4CPM': 'Medicine Unit',
'YC9E': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'H6T': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'Centenary Cardiology': 'Cardiology Unit',
'Centenary Medicine': 'Medicine Unit',
'CPES-IC': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'H3C-SUR (Last Surveyed March 2017)': 'Inpatient Ward',  # Assuming this is a surgical unit
'H6T-Med': 'Medicine Unit',
'H9T (Last Surveyed March 2017)': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'G4FBP (Last Surveyed March 2017)': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'YC4ME': 'Medicine Unit',  # Unsure, mapping to Medicine Unit
'G4D (Last Surveyed March 2017)': 'Inpatient Ward',
'G3A-Sur (previously G4A) (Last Surveyed March 2017)': 'Inpatient Ward',  # Assuming this is a surgical unit
'General Medicine': 'Medicine Unit',
'H2OBS (Last Surveyed March 2017)': 'Inpatient Ward',  # Assuming this is an obstetrics unit
'H2C/H2C-NEPH (Last Surveyed March 2017)': 'Inpatient Ward',  # Assuming this is a nephrology unit
'G4A-Med (previously G3A) (Last Surveyed March 2017)': 'Medicine Unit',
'G4C': 'Inpatient Ward',
'xxH7T-ALC': 'Inpatient Ward',  # Unsure, mapping to Inpatient Ward
'xxH1CP': 'Outpatient',  # Unsure, mapping to Outpatient'
'Belleville General': 'Inpatient Ward', # Assuming this is a hospital/site name
'PrinceEdward county Memorial': 'Inpatient Ward', # Assuming this is a hospital/site name
'North Hastings': 'Inpatient Ward', # Assuming this is a hospital/site name
'Prince Edward County Memorial': 'Inpatient Ward', # Assuming this is a hospital/site name
'Trenton Memorial': 'Inpatient Ward', # Assuming this is a hospital/site name
'Quinte 5 (Acute Medicine)': 'Medicine Unit',
'Q6 Medicine IPU': 'Medicine Unit', # Assuming IPU refers to Inpatient Unit
'Q4 Surgery IPU': 'Inpatient Ward', # Assuming this is a surgical inpatient unit
'Quinte 6 (Surgery)': 'Inpatient Ward', # Assuming this is a surgical unit
'Quinte 4 (ACE/ALC)': 'Inpatient Ward', # Unsure, mapping to Inpatient Ward
'Huntsville District Memorial Hospital': 'Inpatient Ward', # Assuming this is the hospital name/site
'South Muskoka Memorial Hospital': 'Inpatient Ward', # Assuming this is the hospital name/site
'Huntsville District Memorial/Burks Falls Hospital': 'Inpatient Ward', # Assuming this is the hospital name/site
'3WF': 'Inpatient Ward', # Assuming this is an inpatient ward code
'Obstetrics': 'Obstetrics Unit',
'3C': 'Inpatient Ward', # Assuming this is an inpatient ward code
'Uxbridge IP': 'Inpatient Ward', # Assuming IP refers to Inpatient
'Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'3WH': 'Inpatient Ward', # Assuming this is an inpatient ward code
'2E': 'Inpatient Ward', # Assuming this is an inpatient ward code
'CPES-IC - Acute Medicine': 'Medicine Unit',
'Rehab': 'Rehabilitation Unit',
'3E': 'Inpatient Ward', # Assuming this is an inpatient ward code
'CPES-IC - Surgery': 'Inpatient Ward', # Assuming this is a surgical ward
'Paediatrics': 'Pediatrics Unit',
'2C (Last surveyed May 2018)': 'Inpatient Ward', # Assuming this is an inpatient ward code
'1E': 'Inpatient Ward', # Assuming this is an inpatient ward code
'1C': 'Inpatient Ward', # Assuming this is an inpatient ward code
'Uxbridge ER': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'2C': 'Inpatient Ward', # Assuming this is an inpatient ward code
'3WG': 'Inpatient Ward', # Assuming this is an inpatient ward code
'INETEL': 'Outpatient', # Unsure, mapping to Outpatient
'1E (Last surveyed May 2018)': 'Inpatient Ward', # Assuming this is an inpatient ward code
'INEMED': 'Outpatient', # Unsure, mapping to Outpatient
'1C (Last surveyed May 2018)': 'Inpatient Ward', # Assuming this is an inpatient ward code
'MRH C3 MEDICINE AQUA (Mail)': 'Medicine Unit',
'MRH D3 MEDICINE AQUA (Mail)': 'Medicine Unit',
'MRH D4 SURGERY PURPLE (Email)': 'Inpatient Ward',  # Assuming this is a surgical ward
'Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'MRH C4 SURGERY AQUA (Email)': 'Inpatient Ward',  # Assuming this is a surgical ward
'MRH D4 SURGERY PURPLE (Mail)': 'Inpatient Ward',  # Assuming this is a surgical ward
'MRH C3 MEDICINE ORANGE (Mail)': 'Medicine Unit',
'MRH C5 MOTHER (Mail)': 'Obstetrics Unit',  # Assuming this is an obstetrics/maternity ward
'MRH C4 MEDICINE ORANGE (Mail)': 'Medicine Unit',
'MRH C4 SURGERY AQUA (Mail)': 'Inpatient Ward',  # Assuming this is a surgical ward
'MRH C5 MOTHER (Email)': 'Obstetrics Unit',  # Assuming this is an obstetrics/maternity ward
'MRH C3 MEDICINE ORANGE (Email)': 'Medicine Unit',
'MRH D3 MEDICINE AQUA (Email)': 'Medicine Unit',
'MRH C4 MEDICINE PURPLE (Mail)': 'Medicine Unit',
'MRH C4 MEDICINE ORANGE (Email)': 'Medicine Unit',
'MRH C5 L AND D MOTHER (Email)': 'Obstetrics Unit',  # Assuming this is an obstetrics/maternity ward
'MRH C4 MEDICINE PURPLE (Email)': 'Medicine Unit',
'MRH C3 MEDICINE AQUA (Email)': 'Medicine Unit',
'Cardiac Surgery- U6': 'Cardiology Unit',
'U-ER': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Surgery-C51 Urology/ENT': 'Urology Unit',
'V-AED': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Medicine C- D52/D53': 'Medicine Unit',
'Surgery-B91 General Surg': 'Inpatient Ward',  # Assuming this is a general surgical ward
'Surgery-U9 Orthopedic IP': 'Orthopedics Unit',
'Surgery-U8 General Surg': 'Inpatient Ward',  # Assuming this is a general surgical ward
'Medicine-U4': 'Medicine Unit',
'Surgery-D51 Cardiac/Vascular': 'Cardiology Unit',
'CNS-U7': 'Neurology Unit',  # Assuming CNS refers to Central Nervous System
'London Regional Cancer Centre': 'Oncology / Cancer Center',
'Perinatal Care (Mail)': 'Obstetrics Unit',  # Assuming this is related to obstetrics/perinatal care
'Gynecology- D43/D44': 'Gynecology Unit',
'V-PED (ED PPEC)': 'Pediatrics Unit',  # Assuming this is a pediatric emergency unit
'Medicine D- D63/D64': 'Medicine Unit',
'Cardiac- Cardiology U5': 'Cardiology Unit',
'Surgery-B92 Orthopedic Trauma': 'Orthopedics Unit',
'Surgery- C53 Thoracic': 'Inpatient Ward',  # Assuming this is a thoracic surgery ward
'NICU': 'Pediatrics Unit',  # Assuming NICU refers to Neonatal Intensive Care Unit
'Multi Organ Transplant-U-4TU': 'Inpatient Ward',  # Assuming this is a transplant unit
'Oncology- C71/C72/C73/C74': 'Oncology / Cancer Center',
'Nephrology-D62': 'Inpatient Ward',  # Assuming this is a nephrology ward
'Paeds 1': 'Pediatrics Unit',
'xxV-PED': 'Pediatrics Unit',  # Assuming this refers to a pediatric unit
'Paeds 2': 'Pediatrics Unit',
'Medicine B- D71/D72/D73': 'Medicine Unit',
'xxSurgery-C52 Plastics/ENT': 'Inpatient Ward',  # Assuming this is a surgical ward
'Perinatal Care (Phone)': 'Obstetrics Unit',  # Assuming this is related to obstetrics/perinatal care,
'Surgical 4 South': 'Inpatient Ward', # Assuming this is a surgical inpatient ward
'EDPEC': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department', # Unsure, mapping to Emergency Dept.
'Cardiology 8 North': 'Cardiology Unit',
'Surgical 7A &7 B North': 'Inpatient Ward', # Assuming these are surgical inpatient wards
'Medical 4 North': 'Medicine Unit',
'Surgical Short Stay': 'Day Surgery Unit', # Assuming this is a short-stay surgical unit
'Medical 5 South': 'Medicine Unit',
'Medical 6 South': 'Medicine Unit',
'Ambulatory Care': 'Outpatient',
'ACU Clinical Procedures': 'Inpatient Ward', 
'Obstetrics': 'Obstetrics Unit',
'Medical Day Care': 'Outpatient', 
'Day Surgery': 'Day Surgery Unit',
'Juravinski Cancer Centre': 'Oncology / Cancer Center',
'General Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Hamilton General': 'Inpatient Ward',  
'Juranviski': 'Inpatient Ward',  
'McMaster Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Juravinski Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'West Lincoln Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'West Lincoln': 'Inpatient Ward',  
'Inpatient': 'Inpatient Ward',
'Outpatient': 'Outpatient',
'Emergency Department': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Medicine G': 'Inpatient Ward',
'Medicine': 'Medicine Unit',
'Medicine T Telemetry': 'Inpatient Ward',
'Surgery': 'Inpatient Ward',
'Emergency Sarnia': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Rehabilitation': 'Rehabilitation Unit',
'Med/Surg - Rural Health (CEE)': 'Inpatient Ward',
'Obstetrics': 'Obstetrics Unit',
'Surgery B': 'Inpatient Ward',
'Emergency Rural Health': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Day Surgery': 'Day Surgery Unit',
'Medicine A': 'Inpatient Ward',
'Acute Mental Health Inpatient': 'Mental Health',
'Surgery A': 'Inpatient Ward',
'Ophthalmology': 'Ophthalmology Unit',
'Orthopaedics': 'Orthopedics Unit',
'4D': 'Inpatient Ward',
'Emergency': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'6A/6B': 'Inpatient Ward',
'Urology': 'Urology Unit',
'Haematology/Oncology': 'Oncology / Cancer Center',
'5A': 'Inpatient Ward',
'Cardiology': 'Cardiology Unit',
'Diagnostic Imaging': 'Radiology Unit',
'5C': 'Inpatient Ward',
'8A/8B/6E': 'Inpatient Ward',
'5A/5B': 'Inpatient Ward',
'Emergency (Mail)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Acute Care (Mail)': 'Inpatient Ward',
'Emergency (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Acute Care (Phone)': 'Inpatient Ward',
'Respiratory Medicine': 'Respiratory Unit',
'Emergency (Email)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Endocrine': 'Inpatient Ward',
'8C': 'Inpatient Ward',
'Dentistry': 'Dentistry',
'7B/7C/7D/7E': 'Inpatient Ward',
'5B': 'Inpatient Ward',
'Mental Health Outpatient': 'Mental Health',
'Brantford Site': 'Inpatient Ward',
'Ucare TWH': 'Outpatient',
'Fracture Clinic': 'Outpatient',
'Ambulatory': 'Outpatient',
'OP Stroke': 'Outpatient',
'DEC BGH': 'Inpatient Ward',
'DEC-TWH': 'Inpatient Ward',
'5 East': 'Inpatient Ward',
'CHEO. MH OUT-PATIENT SERVICES•6TH FLOOR': 'Mental Health',
'4 West': 'Inpatient Ward',
'CHEO MH OUT-PATIENT SERVICES 6TH FLOOR.': 'Mental Health',
'CHEO MH OUT-PATIENT SERVICES 6TH FLOOR': 'Mental Health',
'CHEO MU OUT-PATIENT SERVICES MONTREAL RD': 'Outpatient',
'EATING DISORDERS INPATIENT PROGRAM CHEO 6': 'Mental Health',
'INPATIENT PSYCHIATRY PROGRAM CHEO 6E': 'Mental Health',
'CHEO MH EATING DISORDER 5 NORTH': 'Mental Health',
'CHEO MH OUT-PATIENT SERVICES MONTREAL RD': 'Mental Health',
'4 North': 'Inpatient Ward',
'CHEO. MFI OUT-PATIENT SERVICES 6TH FLOOR': 'Outpatient',
'.INPATIENT PSYCHIATRY PROGRAM CHEO 6E': 'Mental Health',
'CHEO MH EATING DISORDER :5 NORTH': 'Mental Health',
'CHEO MH OUT-PATIENT SERVICES. MONTREAL RD': 'Mental Health',
'CH EO MN OUT-PATIENT SERVICES MONTREAL RD': 'Outpatient',
'INPATIENT PSYCHIATRY PROGRAM:CHEO 6E': 'Mental Health',
'CHEO MH DAY TREATMENT- CENTRE ADO': 'Mental Health',
'CHEO MH EATING DISORDER .5 NORTH': 'Mental Health',
'CHEO MH EATING DISORDERS NORTH': 'Mental Health',
'CHEO MR OUT-PATIENT SERVICES:6TH FLOOR': 'Outpatient',
'CHEO MH OUT-PATIENT SERVICES:MONTREAL RD': 'Mental Health',
'CHEO MITI OUT-PATIENT SERVICES MONTREAL RD': 'Outpatient',
'EATING DISORDERS INPATIENT pROGRAM CHEO 6': 'Mental Health',
'CHEO MR OUT-PATIENT SERVICES MONTREAL RD': 'Outpatient',
'1CHEO MH OUT-PATIENT:SERVICES 6TH FLOOR': 'Mental Health',
"CHEO MI-I OUT-PATIENT SERVICES 6TH FLOOR'": 'Outpatient',
'CFIE0 MH QUT43ATIENT SERVICES 6TH FLOOR': 'Mental Health',
'EATING DISORDERS. INPATIENT PROGRAM CHEO 6': 'Mental Health',
'.CHEO MH EATING DISORDER 5 NORTH': 'Mental Health',
'CHEO MHOUT-PATIENT SERVICES 6TH FLOOR': 'Mental Health',
'CHEO MI-I OUT-PATIENT SERVICES MONTREAL RD': 'Outpatient',
'CHEO MH OUT-PATIENT SERVICES MONTREAL .RD': 'Mental Health',
'CHEO MH OUT-PATIENTSERVICES MONTREAL RD': 'Mental Health',
'CHEO MH OUT=PATIENT SERVICES 6TH FLOOR': 'Mental Health',
'CHEO .MH OUT-PATIENT SERVICES 6TH FLOOR': 'Mental Health',
'CHEO MH OUT-PATIENT SERVICES. MONTREAL RD.': 'Mental Health',
'CHEO .MH OUT-PATIENT SERVICES MONTREAL RD.': 'Mental Health',
"CHEO MH'OUT-PATIENT SERVICES MONTREAL RD": 'Mental Health',
'CH EO MH OUT-PATIENT. SERVICES MONTREAL RD': 'Mental Health',
'CHEO MH OUT-PATIENT SERVICES MONTREAL RD.': 'Mental Health',
'CHEO MH EATING DISORDER S: NORTH': 'Mental Health',
'CHEO MH OUT-PATIENT SERVICES 6T1-1 FLOOR': 'Mental Health',
'CHEO MI-1 OUT-PATIENT SERVICES. MONTREAL RD': 'Outpatient',
'INPATIENT PSYCHIATRY PROGRAM CHEO GE': 'Mental Health',
'CHEO MH OUT-PATIENT. SERVICES MONTREAL RD': 'Mental Health',
'CHEO MI-I OUT-PATIENT SERVICES .MONTREAL RD': 'Outpatient',
'CHEO MH OUT-PATIENT SERVICES MONTREAL RD:': 'Mental Health',
'CHEO. MH EATING DISORDER 5 NORTH': 'Mental Health',
"CHEO MH OUT-PATIENT SERVICES'6TH FLOOR": 'Mental Health',
'CHEO MO OUT-PATIENT SERVICES 6TH FLOOR': 'Mental Health',
'HEO MI-I OUT-PATIENT SERVICES. MONTREAL RD': 'Outpatient',
'CHEO Mil OUT-PATIENT SERVICES MONTREAL RD': 'Outpatient',
'CHEO MI-I EATING DISORDER 5 NORTH': 'Mental Health',
'EATING DISORDERS INPATIENT PROGRAM CHEO .6.': 'Mental Health',
'CHEO MH OUT PATIENT SERVICES 6TH FLOOR': 'Mental Health',
'CHEO M H OUT-PATIENT SERVICES MONTREAL RD': 'Mental Health',
'Qmpc(Registerecl)': 'Outpatient',
'MHAP': 'Mental Health',
'8AC': 'Inpatient Ward',
'Emergency - CTAS 2 (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'6OC': 'Inpatient Ward',
'6DN': 'Inpatient Ward',
'Emergency - CTAS 4 (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Emergency - CTAS 3 (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Emergency - CTAS 5 (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'6DS': 'Inpatient Ward',
'Emergency -CTAS 4 (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Childrens (Phone)': 'Pediatrics Unit',
'CCC': 'Inpatient Ward', 
'Stroke': 'Inpatient Ward',
'5DS': 'Inpatient Ward',
'5DN': 'Inpatient Ward',
'Emergency -CTAS 5 (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Cancer': 'Oncology / Cancer Center',
'4D': 'Inpatient Ward',
'Medicine': 'Medicine Unit',
'Rehabilitation': 'Rehabilitation Unit',
'5A': 'Inpatient Ward',
'Emergency - CTAS 1 (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Surgery': 'Inpatient Ward',  
'Childbirth': 'Obstetrics Unit',
'Emergency -CTAS 1 (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'4AB': 'Inpatient Ward',
'xxChildrens': 'Pediatrics Unit',
'Emergency (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department',
'Surgery (Phone)': 'Inpatient Ward', 
'Medicine (Phone)': 'Medicine Unit',
'Surgery (Phone)': 'Inpatient Ward', 
'Medicine (Phone)': 'Medicine Unit',
'Emergency (Phone)': 'Emergency Department / Accident and Emergency (A&E) / Casualty Department'
}

In [63]:
unit_mapping['Emergency (Phone)']

'Emergency Department / Accident and Emergency (A&E) / Casualty Department'

In [64]:
def map_unit(value):
    # Check if the value exists in the dictionary
    if value in unit_mapping.keys():
        return unit_mapping[value]
    else:
        return 'Unknown'

In [65]:
df['ProcessedUnit'] = df['Unit'].map(unit_mapping).fillna(df['Unit'])
df['ProcessedUnit2'] = df['Unit'].apply(map_unit)

# Print the DataFrame to see the new "ProcessedUnit" column
df.head()

,Id,Hospital,Type,Comment,CleanedComment,ExperienceDate,ExperienceDateString,Valence,Unit,Code,...,Day,Processed,RefinedCode,Entities,RefinedValence,ProcessedComment,CovidRelated,CovidPeriod,ProcessedUnit,ProcessedUnit2
0,106,Almonte General Hospital,Inpatient,"Compassionate care, such professional staff fr...",compassionate professional staff emergency adm...,2021-02-01 00:00:00+00:00,2/1/2021,Positive,NaN,NaN,...,1,{'parking/transport'},NaN,"[('xxxxxxxxx', 'CHEMICAL')]",Positive,compassionate care such professional staff fro...,False,True,NaN,Unknown
1,430,Almonte General Hospital,Inpatient,I was very pleased with the care & concern giv...,please given thankful health absolutely commen...,2020-06-01 00:00:00+00:00,6/1/2020,Positive,NaN,NaN,...,1,{'parking/transport'},NaN,[],Positive,i wa very pleased with the care concern given ...,False,True,NaN,Unknown
2,650,Almonte General Hospital,Inpatient,Thankfully I was admitted into the last surplu...,admit surplus wsa private washroom button fabu...,2017-08-01 00:00:00+00:00,8/1/2017,Positive,NaN,NaN,...,1,{'parking/transport'},NaN,[],Positive,thankfully i wa admitted into the last surplus...,False,False,NaN,Unknown
3,715,Almonte General Hospital,Inpatient,The nurses and doctors made giving birth durin...,staff staff birth pandemic possible given circ...,2020-05-01 00:00:00+00:00,5/1/2020,Positive,NaN,NaN,...,1,{'parking/transport'},NaN,[],Positive,the nurse and doctor made giving birth during ...,True,True,NaN,Unknown
4,719,Almonte General Hospital,Inpatient,The nurses at the hospital were amazing! Could...,staff thankful,2018-06-01 00:00:00+00:00,6/1/2018,Positive,NaN,NaN,...,1,{'parking/transport'},NaN,[],Positive,the nurse at the hospital were amazing couldn ...,False,False,NaN,Unknown


In [66]:
df.to_csv('Reviews2.csv', index=False)

In [32]:
def getUnitDict(CovidPeriod):
    negative_dict = {}
    positive_dict = {}
    neutral_dict = {}
    posneg_dict = {}
    total_dict = {}
    
    for index, row in df.iterrows():
        if row['CovidPeriod'] == CovidPeriod:
            unit =  str(row['ProcessedUnit2'])

            if unit not in total_dict.keys():
                total_dict[unit] = 1
            elif unit in total_dict.keys():
                total_dict[unit] += 1

            if row['RefinedValence'] == "Negative":
                    if unit not in negative_dict.keys():
                        negative_dict[unit] = 1
                    elif unit in negative_dict.keys():
                        negative_dict[unit] += 1
            elif row['RefinedValence'] == "Positive":
                    if unit not in positive_dict.keys():
                        positive_dict[unit] = 1
                    elif unit in positive_dict.keys():
                        positive_dict[unit] += 1
            elif row['RefinedValence'] == "Neutral":
                    if unit not in neutral_dict.keys():
                        neutral_dict[unit] = 1
                    elif unit in neutral_dict.keys():
                        neutral_dict[unit] += 1
            else:
                    if unit not in posneg_dict.keys():
                        posneg_dict[unit] = 1
                    elif unit in posneg_dict.keys():
                        posneg_dict[unit] += 1

    for unit in negative_dict.keys():
         negative_dict[unit] = round((negative_dict[unit] / total_dict[unit]) * 100, 2)
    
    for unit in positive_dict.keys():
         positive_dict[unit] = round((positive_dict[unit] / total_dict[unit]) * 100, 2)


    return dict(sorted(negative_dict.items(), key=lambda item: item[0])), dict(sorted(positive_dict.items(), key=lambda item: item[0])), neutral_dict, posneg_dict

In [33]:
negative_dict, positive_dict, neutral_dict, posneg_dict = getUnitDict(True)

In [34]:
negative_dict

{'Cardiology Unit': 24.29,
 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)': 17.57,
 'Day Surgery Unit': 18.46,
 'Dentistry': 31.25,
 'Dialysis (Renal) Unit': 15.38,
 'Emergency Department / Accident and Emergency (A&E) / Casualty Department': 32.88,
 'Gynecology Unit': 14.55,
 'Inpatient Ward': 26.47,
 'Medicine Unit': 32.14,
 'Mental Health': 42.79,
 'Neurology Unit': 31.82,
 'Obstetrics Unit': 23.42,
 'Oncology / Cancer Center': 23.26,
 'Ophthalmology Unit': 23.53,
 'Orthopedics Unit': 31.99,
 'Outpatient': 24.14,
 'Pediatrics Unit': 25.84,
 'Radiology Unit': 31.58,
 'Rehabilitation Unit': 23.87,
 'Respiratory Unit': 18.84,
 'Unknown': 30.55,
 'Urology Unit': 30.26}

In [35]:
negative_dict.keys()

dict_keys(['Cardiology Unit', 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)', 'Day Surgery Unit', 'Dentistry', 'Dialysis (Renal) Unit', 'Emergency Department / Accident and Emergency (A&E) / Casualty Department', 'Gynecology Unit', 'Inpatient Ward', 'Medicine Unit', 'Mental Health', 'Neurology Unit', 'Obstetrics Unit', 'Oncology / Cancer Center', 'Ophthalmology Unit', 'Orthopedics Unit', 'Outpatient', 'Pediatrics Unit', 'Radiology Unit', 'Rehabilitation Unit', 'Respiratory Unit', 'Unknown', 'Urology Unit'])

In [36]:
negative_dict.values()

dict_values([24.29, 17.57, 18.46, 31.25, 15.38, 32.88, 14.55, 26.47, 32.14, 42.79, 31.82, 23.42, 23.26, 23.53, 31.99, 24.14, 25.84, 31.58, 23.87, 18.84, 30.55, 30.26])

In [37]:
positive_dict

{'Cardiology Unit': 54.08,
 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)': 63.51,
 'Day Surgery Unit': 65.52,
 'Dentistry': 55.0,
 'Dialysis (Renal) Unit': 66.15,
 'Emergency Department / Accident and Emergency (A&E) / Casualty Department': 42.34,
 'Gynecology Unit': 47.27,
 'Inpatient Ward': 49.29,
 'Medicine Unit': 43.72,
 'Mental Health': 42.36,
 'Neurology Unit': 46.36,
 'Obstetrics Unit': 48.81,
 'Oncology / Cancer Center': 56.07,
 'Ophthalmology Unit': 59.22,
 'Orthopedics Unit': 46.11,
 'Outpatient': 54.18,
 'Pediatrics Unit': 47.69,
 'Radiology Unit': 47.37,
 'Rehabilitation Unit': 54.41,
 'Respiratory Unit': 57.97,
 'Unknown': 46.86,
 'Urology Unit': 49.23}

In [38]:
positive_dict.keys()

dict_keys(['Cardiology Unit', 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)', 'Day Surgery Unit', 'Dentistry', 'Dialysis (Renal) Unit', 'Emergency Department / Accident and Emergency (A&E) / Casualty Department', 'Gynecology Unit', 'Inpatient Ward', 'Medicine Unit', 'Mental Health', 'Neurology Unit', 'Obstetrics Unit', 'Oncology / Cancer Center', 'Ophthalmology Unit', 'Orthopedics Unit', 'Outpatient', 'Pediatrics Unit', 'Radiology Unit', 'Rehabilitation Unit', 'Respiratory Unit', 'Unknown', 'Urology Unit'])

In [39]:
positive_dict.values()

dict_values([54.08, 63.51, 65.52, 55.0, 66.15, 42.34, 47.27, 49.29, 43.72, 42.36, 46.36, 48.81, 56.07, 59.22, 46.11, 54.18, 47.69, 47.37, 54.41, 57.97, 46.86, 49.23])

In [40]:
negative_dict, positive_dict, neutral_dict, posneg_dict = getUnitDict(False)

In [41]:
negative_dict

{'Cardiology Unit': 29.04,
 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)': 20.0,
 'Day Surgery Unit': 27.42,
 'Dentistry': 20.57,
 'Dialysis (Renal) Unit': 19.05,
 'Emergency Department / Accident and Emergency (A&E) / Casualty Department': 39.5,
 'Gynecology Unit': 24.64,
 'Inpatient Ward': 32.45,
 'Medicine Unit': 40.99,
 'Mental Health': 42.86,
 'Neurology Unit': 33.23,
 'Obstetrics Unit': 28.34,
 'Oncology / Cancer Center': 34.34,
 'Ophthalmology Unit': 30.95,
 'Orthopedics Unit': 36.37,
 'Outpatient': 27.65,
 'Pediatrics Unit': 32.36,
 'Radiology Unit': 26.35,
 'Rehabilitation Unit': 33.99,
 'Respiratory Unit': 28.15,
 'Unknown': 35.14,
 'Urology Unit': 32.16}

In [42]:
positive_dict

{'Burn Unit / Burn Center': 87.5,
 'Cardiology Unit': 50.16,
 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)': 62.73,
 'Day Surgery Unit': 55.18,
 'Dentistry': 63.12,
 'Dialysis (Renal) Unit': 47.62,
 'Emergency Department / Accident and Emergency (A&E) / Casualty Department': 35.97,
 'Gynecology Unit': 47.83,
 'Inpatient Ward': 43.65,
 'Medicine Unit': 37.12,
 'Mental Health': 34.49,
 'Neurology Unit': 41.09,
 'Obstetrics Unit': 38.83,
 'Oncology / Cancer Center': 41.06,
 'Ophthalmology Unit': 47.92,
 'Orthopedics Unit': 42.13,
 'Outpatient': 53.86,
 'Pediatrics Unit': 42.24,
 'Radiology Unit': 47.3,
 'Rehabilitation Unit': 45.07,
 'Respiratory Unit': 47.41,
 'Unknown': 40.99,
 'Urology Unit': 49.12}

In [43]:
negative_dict.keys()

dict_keys(['Cardiology Unit', 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)', 'Day Surgery Unit', 'Dentistry', 'Dialysis (Renal) Unit', 'Emergency Department / Accident and Emergency (A&E) / Casualty Department', 'Gynecology Unit', 'Inpatient Ward', 'Medicine Unit', 'Mental Health', 'Neurology Unit', 'Obstetrics Unit', 'Oncology / Cancer Center', 'Ophthalmology Unit', 'Orthopedics Unit', 'Outpatient', 'Pediatrics Unit', 'Radiology Unit', 'Rehabilitation Unit', 'Respiratory Unit', 'Unknown', 'Urology Unit'])

In [44]:
negative_dict.values()

dict_values([29.04, 20.0, 27.42, 20.57, 19.05, 39.5, 24.64, 32.45, 40.99, 42.86, 33.23, 28.34, 34.34, 30.95, 36.37, 27.65, 32.36, 26.35, 33.99, 28.15, 35.14, 32.16])

In [45]:
positive_dict.keys()

dict_keys(['Burn Unit / Burn Center', 'Cardiology Unit', 'Coronary Care Unit (CCU) / Cardiac Intensive Care Unit (CICU)', 'Day Surgery Unit', 'Dentistry', 'Dialysis (Renal) Unit', 'Emergency Department / Accident and Emergency (A&E) / Casualty Department', 'Gynecology Unit', 'Inpatient Ward', 'Medicine Unit', 'Mental Health', 'Neurology Unit', 'Obstetrics Unit', 'Oncology / Cancer Center', 'Ophthalmology Unit', 'Orthopedics Unit', 'Outpatient', 'Pediatrics Unit', 'Radiology Unit', 'Rehabilitation Unit', 'Respiratory Unit', 'Unknown', 'Urology Unit'])

In [46]:
positive_dict.values()

dict_values([87.5, 50.16, 62.73, 55.18, 63.12, 47.62, 35.97, 47.83, 43.65, 37.12, 34.49, 41.09, 38.83, 41.06, 47.92, 42.13, 53.86, 42.24, 47.3, 45.07, 47.41, 40.99, 49.12])

In [125]:
df['Hospital'].unique()

array(['Almonte General Hospital', 'Bluewater',
       'Brant Community Healthcare System', 'Campbellford',
       "Children's Hospital of Eastern Ontario",
       'Dryden Regional Health Centre Comments', 'Erie Shores',
       'Georgian Bay', 'Grand River Hospital', 'Guelph General Hospital',
       'Hamilton', 'Health Sciences North',
       'Holland Bloorview Kids Rehabilitation Centre',
       'Joseph Brant Hospital', 'London', 'Mackenzie Health',
       'Markham Stouffville', 'Mount Sinai',
       'Muskoka Algonquin Healthcare', 'Niagara',
       'Norfolk General Hospital', 'North Bay Regional Health Centre',
       'North York', 'Orillia', 'Ottawa Hospital', 'Pembroke',
       'Peterborough Regional Health Centre',
       'Quinte Health Care Corporation', 'Royal Victoria', 'Sault Area',
       'Scarborough', 'SickKids Hospital', 'Southlake', 'St. Josephs',
       "St. Joseph's Healthcare Hamilton", 'St. Michaels', 'Sunnybrook',
       'Thunder Bay Regional', 'Trillium', 'Universi

In [ ]:

hospital_names = ['Sunnybrook', 'Scarborough', 'Holland Bloorview Kids Rehabilitation Centre', 'Holland Bloorview Kids Rehabilitation Centre', 'SickKids Hospital', 'Mount Sinai', 'University Health Network', 'North York', 'Mackenzie Health', 'Mackenzie Health', 'Markham Stouffville', "Children's Hospital of Eastern Ontario", 'Ottawa Hospital', 'Hamilton', "St. Joseph's Healthcare Hamilton", 'London', 'Grand River Hospital']
filtered_df2 = df[df['Hospital'].isin(hospital_names)]
filtered_df2['Hospital'].unique()
#minority


array(["Children's Hospital of Eastern Ontario", 'Grand River Hospital',
       'Hamilton', 'Holland Bloorview Kids Rehabilitation Centre',
       'London', 'Mackenzie Health', 'Markham Stouffville', 'Mount Sinai',
       'North York', 'Ottawa Hospital', 'Scarborough',
       'SickKids Hospital', "St. Joseph's Healthcare Hamilton",
       'Sunnybrook', 'University Health Network'], dtype=object)

In [8]:

hospital_names = ['Sunnybrook', 'Scarborough', 'Holland Bloorview Kids Rehabilitation Centre', 'Holland Bloorview Kids Rehabilitation Centre', 'SickKids Hospital', 'Mount Sinai', 'University Health Network', 'North York', 'Mackenzie Health', 'Mackenzie Health', 'Markham Stouffville', "Children's Hospital of Eastern Ontario", 'Ottawa Hospital', 'Hamilton', "St. Joseph's Healthcare Hamilton", 'London', 'Grand River Hospital']
filtered_df2 = df[~df['Hospital'].isin(hospital_names)]
filtered_df2['Hospital'].unique()



array(['Almonte General Hospital', 'Bluewater',
       'Brant Community Healthcare System', 'Campbellford',
       'Dryden Regional Health Centre Comments', 'Erie Shores',
       'Georgian Bay', 'Guelph General Hospital', 'Health Sciences North',
       'Joseph Brant Hospital', 'Muskoka Algonquin Healthcare', 'Niagara',
       'Norfolk General Hospital', 'North Bay Regional Health Centre',
       'Orillia', 'Pembroke', 'Peterborough Regional Health Centre',
       'Quinte Health Care Corporation', 'Royal Victoria', 'Sault Area',
       'Southlake', 'St. Josephs', 'St. Michaels', 'Thunder Bay Regional',
       'Trillium', 'Winchester', 'Windsor Regional',
       'Woodstock General Hospital', 'Kingston', 'Michael Garron'],
      dtype=object)

In [ ]:
hospital_names = ['Windsor Regional', 'Kingston', 'St. Michaels', 'SickKids Hospital', 'Mount Sinai', 'University Health Network', 'North York', 'Markham Stouffville', "Children's Hospital of Eastern Ontario", 'Ottawa Hospital', 'Hamilton', 'London', 'Grand River Hospital']
filtered_df3 = df[df['Hospital'].isin(hospital_names)]
filtered_df3['Hospital'].unique()
#low income

array(["Children's Hospital of Eastern Ontario", 'Grand River Hospital',
       'Hamilton', 'London', 'Markham Stouffville', 'Mount Sinai',
       'North York', 'Ottawa Hospital', 'SickKids Hospital',
       'St. Michaels', 'University Health Network', 'Windsor Regional',
       'Kingston'], dtype=object)

In [20]:
hospital_names = ['Windsor Regional', 'Kingston', 'St. Michaels', 'SickKids Hospital', 'Mount Sinai', 'University Health Network', 'North York', 'Markham Stouffville', "Children's Hospital of Eastern Ontario", 'Ottawa Hospital', 'Hamilton', 'London', 'Grand River Hospital']
filtered_df3 = df[~df['Hospital'].isin(hospital_names)]
filtered_df3['Hospital'].unique()

array(['Almonte General Hospital', 'Bluewater',
       'Brant Community Healthcare System', 'Campbellford',
       'Dryden Regional Health Centre Comments', 'Erie Shores',
       'Georgian Bay', 'Guelph General Hospital', 'Health Sciences North',
       'Holland Bloorview Kids Rehabilitation Centre',
       'Joseph Brant Hospital', 'Mackenzie Health',
       'Muskoka Algonquin Healthcare', 'Niagara',
       'Norfolk General Hospital', 'North Bay Regional Health Centre',
       'Orillia', 'Pembroke', 'Peterborough Regional Health Centre',
       'Quinte Health Care Corporation', 'Royal Victoria', 'Sault Area',
       'Scarborough', 'Southlake', 'St. Josephs',
       "St. Joseph's Healthcare Hamilton", 'Sunnybrook',
       'Thunder Bay Regional', 'Trillium', 'Winchester',
       'Woodstock General Hospital', 'Michael Garron'], dtype=object)

In [88]:

hospital_names = ['St. Josephs']
filtered_df2 = df[df['Hospital'].isin(hospital_names)]
filtered_df2['Hospital'].unique()



array(['St. Josephs'], dtype=object)

In [ ]:
St. Josephs, Michael Garron

In [21]:
def getSentimentPercents():
    positive = {}
    negative = {}
    neutral = {}
    posneg = {}
    total = {}

    for index, row in filtered_df3.iterrows():
        year = row["Year"]

        if year not in total:
            total[year] = 1
        else:
            total[year] += 1

        if row['ProcessedValence'] == "Positive":
            if year not in positive:
                positive[year] = 1
            else:
                positive[year] += 1
        elif row['ProcessedValence'] == "Negative":
            if year not in negative:
                negative[year] = 1
            else:
                negative[year] += 1
        elif row['ProcessedValence'] == "Neutral":
            if year not in neutral:
                neutral[year] = 1
            else:
                neutral[year] += 1
        else:
            total[year] -= 1
        
    for year in positive.keys():
         positive[year] = round((positive[year] / total[year]) * 100, 2)
    
    for year in negative.keys():
         negative[year] = round((negative[year] / total[year]) * 100, 2)

    for year in neutral.keys():
         neutral[year] = round((neutral[year] / total[year]) * 100, 2)


    return positive, negative, neutral

In [95]:
def getSentimentPercents2():
        positive = 0
        negative = 0
        neutral = 0
        total = 0

        for index, row in filtered_df2.iterrows():
                if row['ProcessedValence'] == "Positive":
                        positive += 1
                elif row['ProcessedValence'] == "Negative":
                        negative += 1
                elif row['ProcessedValence'] == "Neutral":
                        neutral+= 1
                else:
                        total -= 1
                total += 1

        
        positive = round((positive / total) * 100, 2)

        negative = round((negative / total) * 100, 2)

        neutral = round((neutral / total) * 100, 2)


        return positive, negative, neutral

In [22]:
positive, negative, neutral = getSentimentPercents()

In [96]:
print(getSentimentPercents2())

(47.24, 47.73, 5.04)


In [ ]:
positive #high visible minority

{2021: 56.12,
 2022: 54.96,
 2019: 54.12,
 2018: 51.93,
 2020: 59.93,
 2017: 51.79,
 2023: 57.33,
 2016: 63.48}

In [ ]:
positive #low visible minority

{2021: 59.82,
 2020: 58.32,
 2017: 50.05,
 2018: 51.06,
 2019: 51.35,
 2022: 57.99,
 2015: 64.68,
 2016: 59.06}

In [ ]:
positive #high low income

{2021: 56.71,
 2022: 53.79,
 2019: 55.15,
 2018: 53.6,
 2020: 59.35,
 2017: 53.18,
 2023: 57.33,
 2016: 63.48}

In [ ]:
positive #low low income

{2021: 59.47,
 2020: 58.54,
 2017: 48.83,
 2018: 49.97,
 2019: 50.7,
 2022: 58.42,
 2015: 64.68,
 2016: 59.06}

In [ ]:
negative #high visible minority

{2021: 39.52,
 2022: 40.46,
 2019: 41.77,
 2018: 43.64,
 2020: 35.04,
 2017: 42.86,
 2023: 40.31,
 2016: 30.15}

In [ ]:
negative #low visible minority

{2019: 44.32,
 2018: 44.01,
 2021: 34.94,
 2020: 36.22,
 2017: 44.13,
 2022: 38.73,
 2015: 28.72,
 2016: 35.57}

In [ ]:
negative #high low income

{2021: 38.61,
 2022: 41.51,
 2019: 40.41,
 2018: 42.04,
 2020: 35.52,
 2017: 41.28,
 2023: 40.31,
 2016: 30.15}

In [ ]:
negative #low low income

{2019: 45.13,
 2018: 45.0,
 2021: 35.45,
 2020: 36.04,
 2017: 45.46,
 2022: 38.34,
 2015: 28.72,
 2016: 35.57}

In [ ]:
neutral #high visible minority

{2021: 4.36,
 2022: 4.58,
 2018: 4.43,
 2019: 4.12,
 2017: 5.35,
 2020: 5.02,
 2023: 2.36,
 2016: 6.37}

In [ ]:
neutral #low visible minority

{2020: 5.46,
 2017: 5.82,
 2018: 4.94,
 2021: 5.24,
 2019: 4.33,
 2022: 3.28,
 2015: 6.6,
 2016: 5.37}

In [ ]:
neutral #high low income

{2021: 4.68,
 2022: 4.7,
 2018: 4.36,
 2019: 4.44,
 2017: 5.54,
 2020: 5.13,
 2023: 2.36,
 2016: 6.37}

In [ ]:
neutral #low low income

{2020: 5.43,
 2017: 5.7,
 2018: 5.03,
 2021: 5.08,
 2019: 4.17,
 2022: 3.25,
 2015: 6.6,
 2016: 5.37}

In [124]:
def getSentimentPercents(dataframe):
    total_rows = len(dataframe)
    value_counts = dataframe['RefinedValence'].value_counts()
    percentages = {}
    for valence, count in value_counts.items():
        percentages[valence] = round((count / total_rows) * 100, 2)
    
    return percentages

In [126]:
hospitals = {}

unique_hospitals = df['Hospital'].unique()

for hospital_name in unique_hospitals:
    if str(hospital_name) != "nan":
        percentages = getSentimentPercents(df[df['Hospital'] == hospital_name])
        hospitals[hospital_name] = {'Percentages': percentages}


In [127]:
hospitals

{'Almonte General Hospital': {'Percentages': {'Positive': 87.5,
   'Negative': 12.5}},
 'Bluewater': {'Percentages': {'Positive': 42.16,
   'Negative': 36.33,
   'Positive/Negative': 16.6,
   'Neutral': 4.91}},
 'Brant Community Healthcare System': {'Percentages': {'Positive': 39.49,
   'Negative': 32.67,
   'Positive/Negative': 22.93,
   'Neutral': 4.91}},
 'Campbellford': {'Percentages': {'Positive': 46.48,
   'Negative': 32.6,
   'Positive/Negative': 14.83,
   'Neutral': 6.08}},
 "Children's Hospital of Eastern Ontario": {'Percentages': {'Positive': 43.94,
   'Negative': 30.1,
   'Positive/Negative': 18.34,
   'Neutral': 7.61}},
 'Dryden Regional Health Centre Comments': {'Percentages': {'Positive': 51.25,
   'Negative': 27.64,
   'Positive/Negative': 16.49,
   'Neutral': 4.63}},
 'Erie Shores': {'Percentages': {'Positive': 40.56,
   'Negative': 36.01,
   'Positive/Negative': 17.66,
   'Neutral': 5.77}},
 'Georgian Bay': {'Percentages': {'Positive': 41.76,
   'Negative': 34.1,
   'P